## Import Libraries

In [136]:
import pandas as pd
import plotly.graph_objects as go
from datetime import date
from dash import Dash, dcc, html, Input, Output

## Load Dataset

In [113]:
df1 = pd.read_csv('Coffee Sales_1.csv')
df2 = pd.read_csv('Coffee Sales_2.csv')

df = pd.concat([df1, df2], ignore_index=True)
df.head()

,date,datetime,cash_type,card,money,coffee_name
0,2024-03-01,2024-03-01 10:15:50.520,card,ANON-0000-0000-0001,38.7,Latte
1,2024-03-01,2024-03-01 12:19:22.539,card,ANON-0000-0000-0002,38.7,Hot Chocolate
2,2024-03-01,2024-03-01 12:20:18.089,card,ANON-0000-0000-0002,38.7,Hot Chocolate
3,2024-03-01,2024-03-01 13:46:33.006,card,ANON-0000-0000-0003,28.9,Americano
4,2024-03-01,2024-03-01 13:48:14.626,card,ANON-0000-0000-0004,38.7,Latte


## Inspect the Dataset

In [114]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3373 entries, 0 to 3372
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   date         3373 non-null   object 
 1   datetime     3373 non-null   object 
 2   cash_type    3373 non-null   object 
 3   card         3174 non-null   object 
 4   money        3373 non-null   float64
 5   coffee_name  3373 non-null   object 
dtypes: float64(1), object(5)
memory usage: 158.2+ KB


In [115]:
# Check for missing values
df.isnull().sum()

date             0
datetime         0
cash_type        0
card           199
money            0
coffee_name      0
dtype: int64

In [8]:
# Show rows with missing values
df[df.isnull().any(axis=1)]

,date,datetime,cash_type,card,money,coffee_name
12,2024-03-02,2024-03-02 10:30:35.668,cash,NaN,40.0,Latte
18,2024-03-03,2024-03-03 10:10:43.981,cash,NaN,40.0,Latte
41,2024-03-06,2024-03-06 12:30:27.089,cash,NaN,35.0,Americano with Milk
46,2024-03-07,2024-03-07 10:08:58.945,cash,NaN,40.0,Latte
49,2024-03-07,2024-03-07 11:25:43.977,cash,NaN,40.0,Latte
...,...,...,...,...,...,...
3368,2025-02-24,2025-02-24 18:32:28,card,NaN,28.0,Caramel with Irish whiskey
3369,2025-02-24,2025-02-24 18:33:49,card,NaN,28.0,Caramel with Irish whiskey
3370,2025-02-24,2025-02-24 20:15:01,card,NaN,25.0,Caramel with milk
3371,2025-02-25,2025-02-25 21:41:58,card,NaN,29.0,Coffee with Irish whiskey


In [9]:
# Check for duplicate rows
df.duplicated().sum()

np.int64(0)

## Data Preprocessing

In [134]:
# Convert the 'datetime' column to datetime type
df['datetime'] = pd.to_datetime(df['datetime'], format='ISO8601', errors='coerce')

# Extract the month from the 'datetime' column
df['datetime'].dt.to_period('M')

0       2024-03
1       2024-03
2       2024-03
3       2024-03
4       2024-03
         ...   
3368    2025-02
3369    2025-02
3370    2025-02
3371    2025-02
3372    2025-02
Name: datetime, Length: 3373, dtype: period[M]

## Create Dash app

In [163]:
# Create Dash app
app = Dash(__name__)

## 📊 Total Sales per Coffee Type

In [164]:
app.layout = html.Div(
    children=[
        # DatePickerRange component
        dcc.DatePickerRange(
            id='date-picker-range',
            start_date='2024-01-01',
            end_date=date.today().strftime('%Y-%m-%d'), 
            display_format='YYYY-MM-DD',
            style={ 'height': '100px', 'width': '300px', 'font-size': '10px', 'margin-bottom': '0px'}
        ),
        
        # Bar chart
        dcc.Graph(id='sales-bar-chart')
    ]
)

@app.callback(
    Output('sales-bar-chart', 'figure'),
    Input('date-picker-range', 'start_date'),
    Input('date-picker-range', 'end_date')
)
def update_bar_chart(start_date, end_date):
    # Filter the dataframe based on the selected date range
    filtered_df = df[(df['datetime'] >= start_date) & (df['datetime'] <= end_date)]
    
    # Calculate Total Sales for each Coffee Type
    aggregated_df = filtered_df.groupby('coffee_name', as_index=False)['money'].sum()
    aggregated_df = aggregated_df.sort_values('money', ascending=True)

    # Create the bar chart using go.Bar
    bar_fig = go.Figure(go.Bar(
        x=aggregated_df['money'], 
        y=aggregated_df['coffee_name'], 
        orientation='h', 
        marker=dict(color='rgba(0, 123, 255, 0.3)'),
        hovertemplate='%{x:,.2f}<br>%{y}<extra></extra>'
    ))

    # Update layout with custom styling
    bar_fig.update_layout(
        title={
            'text': 'Total Sales per Coffee Type',
            'x': 0.5, 'xanchor': 'center',
            'font': {'size': 18, 'weight': 'bold'}
        },
        plot_bgcolor='white',  
        paper_bgcolor='white',
        xaxis_title="Total Sales (₴)", 
        xaxis_title_standoff=30,
        yaxis_title="Coffee Type", 
        yaxis_title_standoff=30, 
        height=500,
        font=dict(family="Arial", size=11),
    )

    return bar_fig

# Run the Dash app
app.run_server(mode="inline")